## Daily Impacts

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import time
import timeit
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV
import re

In [3]:
results_scene_1 = pd.read_csv('../results/Final_results/results_scenario1_SRM.csv',index_col=0,usecols=range(6))
results_scene_1.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.000000e+00,5.143610e-01,2.955623e-09,5.216414,0.269225
3-7,1.992068e-07,3.910663e+00,1.179624e-06,30.705659,1.383677
3-9,0.000000e+00,6.564801e-01,4.014125e-07,1.288755,0.054765
3-10,0.000000e+00,6.715375e-01,4.977698e-13,2.220289,0.108173
3-11,0.000000e+00,2.144113e-18,0.000000e+00,3.883684,0.116316


In [4]:
results_scene_2 = pd.read_csv('../results/Final_results/results_scenario2 -$2.75.csv',index_col=0,usecols=range(6))
results_scene_2.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.000000e+00,5.139874e-01,7.175188e-07,5.216787,0.269225
3-7,1.992068e-07,3.910663e+00,1.179624e-06,30.705659,1.383677
3-9,0.000000e+00,6.564801e-01,4.014125e-07,1.288755,0.054765
3-10,0.000000e+00,6.715375e-01,4.977698e-13,2.220289,0.108173
3-11,0.000000e+00,2.144113e-18,0.000000e+00,3.883684,0.116316


In [5]:
results_scene_3 = pd.read_csv('../results/Final_results/results_scenario3 -$10.csv',index_col=0,usecols=range(6))
results_scene_3.head()

,P(mode1),P(mode2),P(mode3),P(mode4),P(mode5)
3-4,0.000000e+00,5.097572e-01,1.991519e-11,5.221018,0.269225
3-7,1.992068e-07,3.910663e+00,1.179624e-06,30.705659,1.383677
3-9,0.000000e+00,6.564801e-01,4.014125e-07,1.288755,0.054765
3-10,0.000000e+00,6.715375e-01,4.977698e-13,2.220289,0.108173
3-11,0.000000e+00,2.144113e-18,0.000000e+00,3.883684,0.116316


In [6]:
wages = pd.read_csv('../modeling/final_allMode_with_wage_cleaned_update.csv')
wages.head(5)

,ODpair,12500,125000,17500,22500,225000,2500,30000,42500,62500,7500,87500,DOlocationID,PUlocationID,duration,mode,nest,price
0,3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,39.695000,2,1,64.000000
1,3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,45.216667,3,1,61.500000
2,3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,83.000000,4,2,5.500000
3,3-4,0.648452,0.740725,0.282211,0.414755,0.514005,0.250819,0.592009,0.671282,0.93859,0.360216,0.586936,4,3,225.933333,5,3,0.000000
4,3-7,3.890709,4.444350,1.693267,2.488532,3.084029,1.504915,3.552056,4.027693,5.63154,2.161294,3.521615,7,3,47.880952,1,1,43.157143


In [7]:
# agg_scene_1 = results_scene_1.drop('destination',axis=1).groupby('origin').sum()
# agg_scene_1.head()
# agg_scene_1.columns = [name + '_predicted' for name in agg_scene_1.columns]
# acs = pd.read_csv('final_acs_transportation_choice.csv',index_col=0)
# acs.head()
# acs.columns = [name + '_actual' for name in acs.columns]
# scene_1_combined = agg_scene_1.merge(acs,how='left',left_index=True,right_index=True)
# scene_1_combined.head()

In [8]:
# Renaming for convenience in calculation
results_scene_1.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_1.columns]
results_scene_2.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_2.columns]
results_scene_3.columns = [re.sub(r'[^\d.]+','',col) for col in results_scene_3.columns]

In [9]:
results_scene_1.reset_index(inplace=True)
results_scene_2.reset_index(inplace=True)
results_scene_3.reset_index(inplace=True)
results_scene_1.rename_axis({'index':'ODpair'},axis=1,inplace=True)
results_scene_2.rename_axis({'index':'ODpair'},axis=1,inplace=True)
results_scene_3.rename_axis({'index':'ODpair'},axis=1,inplace=True)

In [10]:
results_scene_1.head()

,ODpair,1,2,3,4,5
0,3-4,0.000000e+00,5.143610e-01,2.955623e-09,5.216414,0.269225
1,3-7,1.992068e-07,3.910663e+00,1.179624e-06,30.705659,1.383677
2,3-9,0.000000e+00,6.564801e-01,4.014125e-07,1.288755,0.054765
3,3-10,0.000000e+00,6.715375e-01,4.977698e-13,2.220289,0.108173
4,3-11,0.000000e+00,2.144113e-18,0.000000e+00,3.883684,0.116316


In [11]:
# Getting the origin taxi zone
results_scene_1['origin'] = results_scene_1.ODpair.apply(lambda x: x.split('-')[0]).astype('int')
results_scene_2['origin'] = results_scene_2.ODpair.apply(lambda x: x.split('-')[0]).astype('int')
results_scene_3['origin'] = results_scene_3.ODpair.apply(lambda x: x.split('-')[0]).astype('int')

In [12]:
results_scene_1.head()

,ODpair,1,2,3,4,5,origin
0,3-4,0.000000e+00,5.143610e-01,2.955623e-09,5.216414,0.269225,3
1,3-7,1.992068e-07,3.910663e+00,1.179624e-06,30.705659,1.383677,3
2,3-9,0.000000e+00,6.564801e-01,4.014125e-07,1.288755,0.054765,3
3,3-10,0.000000e+00,6.715375e-01,4.977698e-13,2.220289,0.108173,3
4,3-11,0.000000e+00,2.144113e-18,0.000000e+00,3.883684,0.116316,3


In [13]:
# Aggregating the number of people in each ODpair by the origin zone
agg_scene_1 = results_scene_1.groupby('origin').sum()
agg_scene_2 = results_scene_2.groupby('origin').sum()
agg_scene_3 = results_scene_3.groupby('origin').sum()

In [14]:
agg_scene_1.columns = agg_scene_1.columns.astype('int')
agg_scene_2.columns = agg_scene_2.columns.astype('int')
agg_scene_3.columns = agg_scene_3.columns.astype('int')

In [15]:
agg_scene_1.head()

,1,2,3,4,5
origin,,,,,
3,685.603794,977.251252,75.745589,6180.570657,786.828708
4,3702.898406,143.502105,18.156156,4343.766378,2408.676955
5,20.785981,2194.477610,332.314740,5113.791586,566.630083
6,469.347879,833.123340,458.550032,3303.577898,204.400852
7,11600.290406,1758.881372,56.419983,21768.173576,1504.234664


In [16]:
pickup_list = wages.PUlocationID.unique()

In [17]:
origin_time_and_money_avg = wages.groupby(['PUlocationID','mode'],as_index=False)['duration','price'].mean()

In [18]:
origin_time_and_money_avg.head()

,PUlocationID,mode,duration,price
0,3,1,38.302359,37.428045
1,3,2,37.718869,56.582192
2,3,3,41.066196,50.717949
3,3,4,81.756356,5.948623
4,3,5,268.553355,0.000000


In [25]:
agg_scene_1.shape

(256, 5)

In [26]:
len(pickup_list)

256

## Status quo Commute Statistics

In [19]:
pivoted_time = origin_time_and_money_avg.reset_index().pivot(index='PUlocationID',columns='mode',values='duration')

In [20]:
commute_time = pivoted_time.multiply(agg_scene_1).sum().sum()

In [21]:
avg_commute_time = commute_time/agg_scene_1.sum().sum()
avg_commute_time

78.74569460772672

In [22]:
pivoted_money = origin_time_and_money_avg.reset_index().pivot(index='PUlocationID',columns='mode',values='price')

In [23]:
commute_cost = pivoted_money.multiply(agg_scene_1).sum().sum()

In [24]:
avg_commute_cost = commute_cost/agg_scene_1.sum().sum()
avg_commute_cost

13.736955111569769

## Subway ridership

In [47]:
agg_scene_1.sum()[4]

1919490.4661805523

In [48]:
agg_scene_2.sum()[4]

1963122.658801557

In [49]:
agg_scene_3.sum()[4]

2036112.2199620812

### Scenario 1 VS 2

In [41]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_1.columns:
        diff = (agg_scene_2.loc[zone,col]-agg_scene_1.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==col)])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 221.17it/s]


In [42]:
time_diff

1667692.829832572

In [43]:
money_diff

-1286908.224275543

In [44]:
population = agg_scene_1.sum().sum()

In [45]:
time_diff/population

0.5787885396030046

In [46]:
money_diff/population

-0.446633647640205

### Scenario 1 VS 3

In [50]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_1.columns:
        diff = (agg_scene_3.loc[zone,col]-agg_scene_1.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==int(col))])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 226.10it/s]


In [51]:
time_diff

4214488.158604856

In [52]:
money_diff

-3453331.6291632275

In [53]:
population = agg_scene_2.sum().sum()

In [54]:
time_diff/population

1.4626779001849843

In [55]:
money_diff/population

-1.1985113413295143

### Scenario 2 VS 3

In [56]:
time_diff = 0
money_diff = 0
for zone in tqdm(pickup_list,position=0):
    for col in agg_scene_2.columns:
        diff = (agg_scene_3.loc[zone,col]-agg_scene_2.loc[zone,col])
        time_and_money = (origin_time_and_money_avg.loc[(origin_time_and_money_avg['PUlocationID']==zone)\
                                                       & (origin_time_and_money_avg['mode']==int(col))])
        time_diff+= diff * (0 if time_and_money.empty else time_and_money['duration'].values[0])
        money_diff+= diff * (0 if time_and_money.empty else time_and_money['price'].values[0])

100%|██████████| 256/256 [00:01<00:00, 219.80it/s]


In [57]:
time_diff

2546795.3287722874

In [58]:
money_diff

-2166423.404887686

In [59]:
population = agg_scene_2.sum().sum()

In [60]:
time_diff/population

0.883889360581981

In [367]:
money_diff/population

-0.7518776936893098

#### Use below code only if you want exact values for the difference in time/money for commuters as it takes a very long time to run (>3 hours). It can also be used to get the difference for each taxi zone (after some minor changes).

In [148]:
# time_total = 0
# money_total = 0
# for pair in tqdm(ODpair_list,position=0):
#     time_pair = 0
#     money_pair = 0
#     for col in results_scene_1.columns:
#         time_pair+=(results_scene_1.loc[results_scene_1.index==pair,col]-results_scene_2.loc[results_scene_2.index==pair,col])\
#         *wages.loc[(wages['ODpair']==pair) & wages['mode']==int(col),'duration']
#         money_pair+=(results_scene_1.loc[results_scene_1.index==pair,col]-results_scene_2.loc[results_scene_2.index==pair,col])\
#         *wages.loc[(wages['ODpair']==pair) & wages['mode']==int(col),'price']
#     time_total+=time_pair
#     money_total+=money_pair